# 한글 텍스트 처리 - 네이버 영화 평점 감성 분석 

### 한글 NLP 처리의 어려움
* 띄어쓰기 : 영어의 띄어쓰기 경우 의미 왜곡이 아닌 잘못된 또는 없는 단어로 인식되지만, 한국어는 잘못된 띄어쓰기로 의미가 왜곡된다.
* 다양한 조사 : 조사는 주어나 목적어를 위해 추가되는데, 경우의 수가 너무 많아 어근 추출(Stemming/Lemmatization) 등의 전처리 시 제거하기가 까다롭다.

### KoNLPy - 파이썬 기반의 한글 형태소 패키지
* 형태소 : 단어로서 의미를 가지는 최소 단위
* 형태소 분석 : 형태소 어근 단위로 쪼개고 각 형태소에 품사 태깅(POS tagging)을 부착하는 작업
* 형태소 분석 모듈 : 꼬꼬마(Kkma), 한나눔(Hannanum), Komoran, 은전한닢 프로젝트(Mecab) - 리눅스 환경에서만 가능, Twitter => Mecab이 가장 뛰어남


In [1]:
import warnings
warnings.filterwarnings('ignore')

#### KoNLPy 설치

In [2]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip install konlpy

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [3]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd-64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd-64"


#### mecab 설치

    %%bash
    bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
    pip3 install /tmp/mecab-python-0.996

### 데이터 로딩
* 네이버 영화 평점 데이터 : https://github.com/e9t/nsmc

In [4]:
import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/파이썬_머신러닝_완벽가이드/08_Text_Analytics/naver_movie_data/ratings_train.txt', sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


* 학습 데이터 세트의 0과 1의 Label 값 비율을 살펴보면 1이 긍정, 0이 부정 감성.

In [5]:
train_df['label'].value_counts()

0    75173
1    74827
Name: label, dtype: int64

0과 1의 비율이 어느 한쪽으로 치우치지 않고 균등한 분포를 나타내고 있다. 

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


train_df의 경우 리뷰 텍스트를 가지는 'document' 칼럼에 Null이 일부 존재한다.  문자가 아닌 숫자의 경우 단어적인 의미로 부족하므로 정규 표현식을 사용하여 공백으로 변환.

In [7]:
import re

train_df = train_df.fillna(' ')
# \d = 숫자
train_df['document'] = train_df['document'].apply(lambda x : re.sub(r"\d+", " ", x))

# 테스트 데이터 세트를 로딩하고 동일하게 Null 및 숫자를 공백으로 변환
test_df = pd.read_csv('/content/drive/MyDrive/파이썬_머신러닝_완벽가이드/08_Text_Analytics/naver_movie_data/ratings_test.txt', sep='\t')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply(lambda x : re.sub(r"\d+", " ", x))

# id 칼럼 삭제
train_df.drop('id', axis=1, inplace=True)
test_df.drop('id', axis=1, inplace=True)

* 각 문장을 한글 형태소 분석을 통해 형태소 단어로 토큰화 - SNS 분석에 적합한 Twitter 클래스 이용.
* Twitter 객체의 morphs() 메서드를 이용 - 입력 인자로 들어온 문장을 형태로 단어 형태로 토큰화해 list 객체로 반환.
* TF-IDF를 이용하여 단어 피처 벡터화.


In [8]:
from konlpy.tag import Okt # Twitter changed Okt

twitter = Okt()
def tw_tokenizer(text):
    # 입력 인자로 들어온 텍스트를 형태소 단어로 토큰화해 리스트 형태로 반환
    tokens_ko = twitter.morphs(text)
    return tokens_ko

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Twitter 객체의 morphs() 객체를 이용한 tokenizer를 사용. ngram_range는(1, 2)
tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1, 2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

In [10]:
# 로지스틱 회귀를 이용해 분류 기반의 감성 분석 수행
lg_clf = LogisticRegression(random_state=0)

# 파라미터 C 최적화를 위해 GridSearchCv를 이용
params = { 'C': [1 ,3.5, 4.5, 5.5, 10 ] }
grid_cv = GridSearchCV(lg_clf , param_grid=params , cv=3 ,scoring='accuracy', verbose=1 )
grid_cv.fit(tfidf_matrix_train , train_df['label'] )
print(grid_cv.best_params_ , round(grid_cv.best_score_,4))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
{'C': 3.5} 0.8593


C가 3.5일 때 최고 0.8693의 정확도를 보인다.

테스트 세트를 이용해 최종 감성 분석 예측을 수행.

In [11]:
from sklearn.metrics import accuracy_score

# 테스트 세트를 이용해 에측할 때는 학습할 때 적용한 TfidfVectorizer를 그대로 사용해야 한다.
# 학습 시 설정된 TfidfVectorizer의 피처 개수와 테스트 데이터를 TfidfVectorizer로 변환할 피처 개수가 같아진다.
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

# classifier는 GridSearchCV에서 최적 파라미터로 학습된 classifier를 그대로 이용
best_estimator = grid_cv.best_estimator_
preds = best_estimator.predict(tfidf_matrix_test)

print('Logistic Regression 정확도: ',accuracy_score(test_df['label'],preds))

Logistic Regression 정확도:  0.86186
